In [1]:
import time
from collections import OrderedDict

import tmdbsimple as tmdb # https://www.themoviedb.org/documentation/api
tmdb.API_KEY = ''
import imdb

from modules import IO

In [2]:
data_folder = 'data/'
n_tmdb = 3000
n_top = 1000

In [3]:
tmdb_genres = tmdb.Genres().movie_list()
old_ids = []
tmdb_genres_list = []
for g in tmdb_genres['genres']:
    old_ids.append(g['id'])
    tmdb_genres_list.append(g['name'])
old_id2new_id = {_i:i for i, _i in enumerate(old_ids)}
IO(data_folder + 'tmdb_genres_list.pkl').to_pickle(tmdb_genres_list)



In [4]:
tmdb_genres_list = IO(data_folder + 'tmdb_genres_list.pkl').read_pickle()

print(len(tmdb_genres_list))
tmdb_genres_list

19


['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

In [5]:
%%time

movies = tmdb.Movies()
top_tmdb_raw = []
for i in range(1, n_tmdb // 20 + 1):
    top_tmdb_raw.extend(movies.popular(page=i)['results'])
    if i % 10 == 0:
        print(i)
        time.sleep(3)

for m in top_tmdb_raw:
    m['genre_ids'] = [old_id2new_id[i] for i in m['genre_ids']]

IO(data_folder + 'top{}_tmdb_raw.pkl'.format(n_tmdb)).to_pickle(top_tmdb_raw)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


In [5]:
top_tmdb_raw = IO(data_folder + 'top{}_tmdb_raw.pkl'.format(n_tmdb)).read_pickle()
print(len(top_tmdb_raw))
top_tmdb_raw[0]

3000


{'adult': False,
 'backdrop_path': '/9ywA15OAiwjSTvg3cBs9B7kOCBF.jpg',
 'genre_ids': [6, 13],
 'id': 337167,
 'original_language': 'en',
 'original_title': 'Fifty Shades Freed',
 'overview': 'Believing they have left behind shadowy figures from their past, newlyweds Christian and Ana fully embrace an inextricable connection and shared life of luxury. But just as she steps into her role as Mrs. Grey and he relaxes into an unfamiliar stability, new threats could jeopardize their happy ending before it even begins.',
 'popularity': 592.721736,
 'poster_path': '/jjPJ4s3DWZZvI4vw8Xfi4Vqa1Q8.jpg',
 'release_date': '2018-02-07',
 'title': 'Fifty Shades Freed',
 'video': False,
 'vote_average': 6.2,
 'vote_count': 1009}

In [6]:
ia = imdb.IMDb()

In [7]:
%%time
top = []
imdb_info = ['title', 'genres', 'plot', 'synopsis', 'rating', 'votes']
i = 0

for v in top_tmdb_raw:
    title = v['title']
    try:
        movie = ia.search_movie(title)[0]
        ia.update(movie)
    except:
        continue
    if title == movie['title']:
        keep = True
        for it in imdb_info:
            if not it in movie.keys():
                keep = False
                break
        if not keep:
            continue
        top.append(OrderedDict([('tmdb_title', v['title']), \
                                ('tmdb_genre_ids', v['genre_ids']), \
                                ('tmdb_genres', [tmdb_genres_list[g_id] for g_id in v['genre_ids']]), 
                                ('tmdb_overview', v['overview']), \
                                ('tmdb_rating', v['vote_average']), \
                                ('tmdb_votes', v['vote_count'])]))
        for it in imdb_info:
            top[i]['imdb_' + it] = movie[it]
        
        print('{}. {}; {}'.format(i + 1, top[i]['tmdb_title'], top[i]['imdb_title']))
        i += 1
        if i == n_top:
            break
    time.sleep(0.2)
    
IO(data_folder + 'top{}_all.pkl'.format(n_top)).to_pickle(top)

1. Fifty Shades Freed; Fifty Shades Freed
2. Zootopia; Zootopia
3. Black Panther; Black Panther
4. Coco; Coco
5. Star Wars: The Last Jedi; Star Wars: The Last Jedi
6. The Shape of Water; The Shape of Water
7. Tomb Raider; Tomb Raider
8. Thor: Ragnarok; Thor: Ragnarok
9. Justice League; Justice League
10. Annihilation; Annihilation
11. Beauty and the Beast; Beauty and the Beast
12. The Maze Runner; The Maze Runner
13. Jumanji: Welcome to the Jungle; Jumanji: Welcome to the Jungle
14. John Wick; John Wick
15. Sleight; Sleight
16. Blade Runner 2049; Blade Runner 2049
17. Guardians of the Galaxy; Guardians of the Galaxy
18. Teenage Mutant Ninja Turtles; Teenage Mutant Ninja Turtles
19. Avengers: Age of Ultron; Avengers: Age of Ultron
20. Thor: The Dark World; Thor: The Dark World
21. The Avengers; The Avengers
22. The Hunger Games: Mockingjay - Part 1; The Hunger Games: Mockingjay - Part 1
23. Pirates of the Caribbean: The Curse of the Black Pearl; Pirates of the Caribbean: The Curse of th

189. Elysium; Elysium
190. The Great Gatsby; The Great Gatsby
191. Mission: Impossible - Ghost Protocol; Mission: Impossible - Ghost Protocol
192. How to Train Your Dragon; How to Train Your Dragon
193. GoldenEye; GoldenEye
194. The Chronicles of Narnia: The Lion, the Witch and the Wardrobe; The Chronicles of Narnia: The Lion, the Witch and the Wardrobe
195. The World Is Not Enough; The World Is Not Enough
196. The Godfather: Part II; The Godfather: Part II
197. Aliens; Aliens
198. Now You See Me 2; Now You See Me 2
199. Memento; Memento
200. Spider-Man; Spider-Man
201. The Shawshank Redemption; The Shawshank Redemption
202. The Expendables; The Expendables
203. Blade II; Blade II
204. Shrek 2; Shrek 2
205. Wrong Turn 2: Dead End; Wrong Turn 2: Dead End
206. Godzilla; Godzilla
207. WALL·E; WALL·E
208. Tangled; Tangled
209. Oblivion; Oblivion
210. Men in Black II; Men in Black II
211. Maleficent; Maleficent
212. Reservoir Dogs; Reservoir Dogs
213. Night at the Museum; Night at the Museu

407. The Nice Guys; The Nice Guys
408. Lincoln; Lincoln
409. Looper; Looper
410. From Dusk Till Dawn; From Dusk Till Dawn
411. Fantastic 4: Rise of the Silver Surfer; Fantastic 4: Rise of the Silver Surfer
412. Live Free or Die Hard; Live Free or Die Hard
413. The Heat; The Heat
414. Crash; Crash
415. Black Mass; Black Mass
416. The Tree of Life; The Tree of Life
417. Bridget Jones's Diary; Bridget Jones's Diary
418. Rogue One: A Star Wars Story; Rogue One: A Star Wars Story
419. Night at the Museum: Battle of the Smithsonian; Night at the Museum: Battle of the Smithsonian
420. The Girl with the Dragon Tattoo; The Girl with the Dragon Tattoo
421. Kingsman: The Secret Service; Kingsman: The Secret Service
422. This Is the End; This Is the End
423. War of the Worlds; War of the Worlds
424. The Mummy: Tomb of the Dragon Emperor; The Mummy: Tomb of the Dragon Emperor
425. Madagascar 3: Europe's Most Wanted; Madagascar 3: Europe's Most Wanted
426. Transporter 3; Transporter 3
427. Predators

618. Underworld: Blood Wars; Underworld: Blood Wars
619. The Hurt Locker; The Hurt Locker
620. Airplane!; Airplane!
621. The Terminal; The Terminal
622. Edge of Tomorrow; Edge of Tomorrow
623. Blade; Blade
624. Annie Hall; Annie Hall
625. Natural Born Killers; Natural Born Killers
626. National Treasure: Book of Secrets; National Treasure: Book of Secrets
627. Eternal Sunshine of the Spotless Mind; Eternal Sunshine of the Spotless Mind
628. Lawrence of Arabia; Lawrence of Arabia
629. The Jungle Book; The Jungle Book
630. The Expendables 3; The Expendables 3
631. Flatliners; Flatliners
632. Ghost; Ghost
633. Dark City; Dark City
634. Friends with Benefits; Friends with Benefits
635. Mystic River; Mystic River
636. Rocky II; Rocky II
637. The Dark Tower; The Dark Tower
638. Identity; Identity
639. The Girl with the Dragon Tattoo; The Girl with the Dragon Tattoo
640. The Graduate; The Graduate
641. The Hunger Games: Catching Fire; The Hunger Games: Catching Fire
642. The Family; The Famil

826. Boogie Nights; Boogie Nights
827. Annie; Annie
828. The Descendants; The Descendants
829. Bad Boys; Bad Boys
830. Grindhouse; Grindhouse
831. As Good as It Gets; As Good as It Gets
832. Paranormal Activity; Paranormal Activity
833. Marley & Me; Marley & Me
834. The Machinist; The Machinist
835. Heat; Heat
836. The Name of the Rose; The Name of the Rose
837. The Fault in Our Stars; The Fault in Our Stars
838. Let's Be Cops; Let's Be Cops
839. The Ugly Truth; The Ugly Truth
840. The Best Offer; The Best Offer
841. Memoirs of a Geisha; Memoirs of a Geisha
842. Hitch; Hitch
843. O Brother, Where Art Thou?; O Brother, Where Art Thou?
844. Paul; Paul
845. Wedding Crashers; Wedding Crashers
846. Independence Day; Independence Day
847. The Other Guys; The Other Guys
848. Death Race 2; Death Race 2
849. The Circle; The Circle
850. A Walk Among the Tombstones; A Walk Among the Tombstones
851. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb; Dr. Strangelove or: How I Lea

In [9]:
keys = ['tmdb_title', 'tmdb_genre_ids', 'tmdb_genres', 'tmdb_overview', 'tmdb_rating', 'tmdb_votes', \
        'imdb_title', 'imdb_genre_ids', 'imdb_genres', 'imdb_plot', 'imdb_synopsis', 'imdb_rating', 'imdb_votes']

imdb_genres_list = set()

for v in top:
    for g in v['imdb_genres']:
        imdb_genres_list.add(g)
        
imdb_genres_list = list(imdb_genres_list)
IO(data_folder + 'imdb_genres_list.pkl').to_pickle(imdb_genres_list)

imdb_g2id = {v:i for i, v in enumerate(imdb_genres_list)}

for i, v in enumerate(top):
    v['imdb_genre_ids'] = [imdb_g2id[g] for g in v['imdb_genres']]
    top[i] = OrderedDict((key, v[key]) for key in keys)

IO(data_folder + 'top{}_all.pkl'.format(n_top)).to_pickle(top)
    
imdb_genres_list = IO(data_folder + 'imdb_genres_list.pkl').read_pickle()
print(len(imdb_genres_list))
imdb_genres_list

20


['Drama',
 'Crime',
 'Romance',
 'Animation',
 'Mystery',
 'Sci-Fi',
 'Biography',
 'Action',
 'Western',
 'Fantasy',
 'Musical',
 'Sport',
 'Adventure',
 'Thriller',
 'Family',
 'Music',
 'War',
 'Horror',
 'History',
 'Comedy']